# Joseph D. Piotroski의 F-Score

In [ ]:
# ROA (Return on Assets): 양수면 1점
# CFO (Cash Flow from Operations): 양수면 1점
# ROA의 변화: 전년 대비 증가하면 1점
# CFO가 ROA를 초과하는 경우: 초과하면 1점
# 레버리지 비율의 변화: 감소하면 1점
# 유동성 비율의 변화: 증가하면 1점
# 발행 주식의 변화: 증가하면 0점, 그롷치 않으면 1점
# 매출총이익률 (Gross Margin) 변화: 전년 대비 증가하면 1점
# 자산회전율 (Asset Turnover) 변화: 전년 대비 증가하면 1점

## 기업의 재무 데이터를 가져오기

In [4]:
pip install dart-fss


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: py in c:\users\jyb\appdata\local\programs\python\python39\lib\site-packages (from dart-fss) (1.11.0)



In [ ]:
## DART API 키 설정

In [ ]:
import dart_fss as dart

# DART API 키 설정
api_key = 'YOUR_DART_API_KEY'
dart.set_api_key(api_key)


In [ ]:
## 코스닥 가져오기

In [ ]:
# 코스닥 시장에 상장된 기업 목록 가져오기
corp_list = dart.get_corp_list(market="KOSDAQ")

In [ ]:
##  재무 데이터 가져오기

In [ ]:
import pandas as pd

# 특정 연도의 재무 데이터를 가져오는 함수
def get_financial_data(corp, year):
    fs = corp.extract_fs(bgn_de=f'{year}0101')
    data = {
        'ticker': corp.stock_code,
        'year': year,
        'roa': fs.get('roa', None),
        'cfo': fs.get('cfo', None),
        'leverage': fs.get('liabilities_to_assets', None),
        'current_ratio': fs.get('current_ratio', None),
        'shares_outstanding': corp.get_number_of_shares(),
        'gross_margin': fs.get('gross_margin', None),
        'asset_turnover': fs.get('asset_turnover', None)
    }
    return pd.Series(data)

# 기업별 재무 데이터 가져오기
year = 2022
financial_data = pd.DataFrame([get_financial_data(corp, year) for corp in corp_list])


## F-Score 계산

In [ ]:
def calculate_f_score(data):
    f_score = 0

    # ROA > 0
    if data['roa'] > 0:
        f_score += 1
    
    # CFO > 0
    if data['cfo'] > 0:
        f_score += 1
    
    # ROA의 변화 > 0 (전년도 데이터 필요)
    prev_year_data = get_financial_data(dart.find_corp(data['ticker']), data['year'] - 1)
    if data['roa'] > prev_year_data['roa']:
        f_score += 1
    
    # CFO > ROA
    if data['cfo'] > data['roa'] * prev_year_data['asset_turnover']:
        f_score += 1
    
    # 레버리지 비율 감소
    if data['leverage'] < prev_year_data['leverage']:
        f_score += 1
    
    # 유동성 비율 증가
    if data['current_ratio'] > prev_year_data['current_ratio']:
        f_score += 1
    
    # 발행 주식 감소
    if data['shares_outstanding'] <= prev_year_data['shares_outstanding']:
        f_score += 1
    
    # 매출총이익률 증가
    if data['gross_margin'] > prev_year_data['gross_margin']:
        f_score += 1
    
    # 자산회전율 증가
    if data['asset_turnover'] > prev_year_data['asset_turnover']:
        f_score += 1
    
    return f_score

# F-Score 계산
financial_data['f_score'] = financial_data.apply(calculate_f_score, axis=1)


## 백테스팅 수행

In [ ]:
import numpy as np
from datetime import datetime

# 백테스팅 수행 (단순 예제)
def backtest(data, start_date, end_date):
    portfolio_returns = []

    for year in range(start_date.year, end_date.year + 1):
        # 해당 연도의 F-Score 상위 종목 선택
        year_data = data[data['year'] == year]
        top_stocks = year_data.sort_values('f_score', ascending=False).head(10)
        
        # 종목별 연간 수익률 계산 (이 예제에서는 가정된 수익률 사용)
        top_stock_returns = np.random.normal(0.1, 0.05, len(top_stocks))  # 예제 데이터
        
        portfolio_returns.append(np.mean(top_stock_returns))
    
    return portfolio_returns

# 백테스트 기간 설정
start_date = datetime(2015, 1, 1)
end_date = datetime(2023, 12, 31)

# 백테스팅 실행
portfolio_returns = backtest(financial_data, start_date, end_date)
cumulative_return = np.prod([1 + r for r in portfolio_returns]) - 1

print(f"백테스트 기간 동안 누적 수익률: {cumulative_return:.2f}")


In [1]:
import pandas as pd

def calculate_f_score(data):
    """
    피오트로스키의 F-Score를 계산하는 함수.
    
    :param data: 재무 데이터가 포함된 데이터프레임 (index는 연도, columns는 해당 지표들)
    :return: 각 연도별 F-Score를 포함한 데이터프레임
    """
    f_score = pd.DataFrame(index=data.index)
    
    # 수익성 지표
    f_score['ROA'] = (data['net_income'] / data['total_assets']).apply(lambda x: 1 if x > 0 else 0)
    f_score['CFO'] = data['cash_flow_operations'].apply(lambda x: 1 if x > 0 else 0)
    f_score['change_ROA'] = (data['net_income'] / data['total_assets']).diff().apply(lambda x: 1 if x > 0 else 0)
    
    # 레버리지, 유동성, 자금조달 지표
    f_score['change_leverage'] = (data['long_term_debt'] / data['total_assets']).diff().apply(lambda x: 1 if x < 0 else 0)
    f_score['change_liquidity'] = (data['current_assets'] / data['current_liabilities']).diff().apply(lambda x: 1 if x > 0 else 0)
    f_score['equity_offering'] = data['shares_outstanding'].diff().apply(lambda x: 1 if x <= 0 else 0)
    
    # 운영 효율성 지표
    f_score['change_margin'] = (data['gross_margin'] / data['total_revenue']).diff().apply(lambda x: 1 if x > 0 else 0)
    f_score['change_turnover'] = (data['total_revenue'] / data['total_assets']).diff().apply(lambda x: 1 if x > 0 else 0)
    
    f_score['F_Score'] = f_score.sum(axis=1)
    
    return f_score

# 백테스팅을 위한 예제 사용법
data = pd.DataFrame({
    'net_income': [100, 150, 200],
    'total_assets': [1000, 1100, 1200],
    'cash_flow_operations': [120, 130, 140],
    'long_term_debt': [400, 350, 300],
    'current_assets': [500, 550, 600],
    'current_liabilities': [200, 220, 250],
    'shares_outstanding': [1000, 1000, 1000],
    'gross_margin': [400, 450, 500],
    'total_revenue': [1000, 1100, 1200]
}, index=[2018, 2019, 2020])

f_score_df = calculate_f_score(data)
print(f_score_df)


      ROA  CFO  change_ROA  change_leverage  change_liquidity  \
2018    1    1           0                0                 0   
2019    1    1           1                1                 0   
2020    1    1           1                1                 0   

      equity_offering  change_margin  change_turnover  F_Score  
2018                0              0                0        2  
2019                1              1                0        6  
2020                1              1                0        6  


In [3]:
import pandas as pd
import numpy as np

def calculate_f_score(data):
    f_score = pd.DataFrame(index=data.index)
    f_score['ROA'] = (data['net_income'] / data['total_assets']).apply(lambda x: 1 if x > 0 else 0)
    f_score['CFO'] = data['cash_flow_operations'].apply(lambda x: 1 if x > 0 else 0)
    f_score['change_ROA'] = (data['net_income'] / data['total_assets']).diff().apply(lambda x: 1 if x > 0 else 0)
    f_score['change_leverage'] = (data['long_term_debt'] / data['total_assets']).diff().apply(lambda x: 1 if x < 0 else 0)
    f_score['change_liquidity'] = (data['current_assets'] / data['current_liabilities']).diff().apply(lambda x: 1 if x > 0 else 0)
    f_score['equity_offering'] = data['shares_outstanding'].diff().apply(lambda x: 1 if x <= 0 else 0)
    f_score['change_margin'] = (data['gross_margin'] / data['total_revenue']).diff().apply(lambda x: 1 if x > 0 else 0)
    f_score['change_turnover'] = (data['total_revenue'] / data['total_assets']).diff().apply(lambda x: 1 if x > 0 else 0)
    f_score['F_Score'] = f_score.sum(axis=1)
    return f_score

def backtest_strategy(price_data, f_score_df, threshold=5):
    returns = price_data.pct_change().shift(-1).fillna(0)
    portfolio_returns = pd.Series(index=returns.index)

    for year in f_score_df.index[:-1]:
        selected_stocks = f_score_df.loc[year][f_score_df.loc[year] >= threshold].index
        if len(selected_stocks) > 0:
            selected_stocks = [stock for stock in selected_stocks if stock in returns.columns]
            if selected_stocks:
                portfolio_returns.loc[str(year+1)] = returns[selected_stocks].mean(axis=1)
    
    portfolio_cumulative_returns = (1 + portfolio_returns.dropna()).cumprod() - 1
    return portfolio_returns.dropna(), portfolio_cumulative_returns

def calculate_sharpe_ratio(returns, risk_free_rate=0.02):
    excess_returns = returns - risk_free_rate / 252  # 일일 수익률로 가정
    sharpe_ratio = np.mean(excess_returns) / np.std(excess_returns)
    return sharpe_ratio * np.sqrt(252)  # 연간 샤프비율로 변환

# 예제 사용법
data = pd.DataFrame({
    'net_income': [100, 150, 200],
    'total_assets': [1000, 1100, 1200],
    'cash_flow_operations': [120, 130, 140],
    'long_term_debt': [400, 350, 300],
    'current_assets': [500, 550, 600],
    'current_liabilities': [200, 220, 250],
    'shares_outstanding': [1000, 1000, 1000],
    'gross_margin': [400, 450, 500],
    'total_revenue': [1000, 1100, 1200]
}, index=[2018, 2019, 2020])

price_data = pd.DataFrame({
    'AAPL': [150, 155, 160, 165, 170, 175, 180],
    'MSFT': [100, 102, 104, 106, 108, 110, 112],
    'GOOGL': [1200, 1220, 1240, 1260, 1280, 1300, 1320]
}, index=pd.date_range(start='2020-01-01', periods=7, freq='Y'))

f_score_df = calculate_f_score(data)
f_score_df['AAPL'] = [6, 5, 7]
f_score_df['MSFT'] = [4, 6, 5]
f_score_df['GOOGL'] = [7, 8, 6]

portfolio_returns, portfolio_cumulative_returns = backtest_strategy(price_data, f_score_df)

# 샤프비율 계산
sharpe_ratio = calculate_sharpe_ratio(portfolio_returns)
print(f"Sharpe Ratio: {sharpe_ratio}")

print("Portfolio Cumulative Returns:")
print(portfolio_cumulative_returns)


C:\Users\JYB\AppData\Local\Temp/ipykernel_24772/2217281878.py:19: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  portfolio_returns = pd.Series(index=returns.index)


TypeError: cannot convert the series to <class 'float'>

In [4]:
import pandas as pd
import numpy as np

def calculate_f_score(data):
    f_score = pd.DataFrame(index=data.index)
    f_score['ROA'] = (data['net_income'] / data['total_assets']).apply(lambda x: 1 if x > 0 else 0)
    f_score['CFO'] = data['cash_flow_operations'].apply(lambda x: 1 if x > 0 else 0)
    f_score['change_ROA'] = (data['net_income'] / data['total_assets']).diff().apply(lambda x: 1 if x > 0 else 0)
    f_score['change_leverage'] = (data['long_term_debt'] / data['total_assets']).diff().apply(lambda x: 1 if x < 0 else 0)
    f_score['change_liquidity'] = (data['current_assets'] / data['current_liabilities']).diff().apply(lambda x: 1 if x > 0 else 0)
    f_score['equity_offering'] = data['shares_outstanding'].diff().apply(lambda x: 1 if x <= 0 else 0)
    f_score['change_margin'] = (data['gross_margin'] / data['total_revenue']).diff().apply(lambda x: 1 if x > 0 else 0)
    f_score['change_turnover'] = (data['total_revenue'] / data['total_assets']).diff().apply(lambda x: 1 if x > 0 else 0)
    f_score['F_Score'] = f_score.sum(axis=1)
    return f_score

def backtest_strategy(price_data, f_score_df, threshold=5):
    returns = price_data.pct_change().shift(-1).fillna(0)
    portfolio_returns = pd.Series(index=returns.index)

    for year in f_score_df.index[:-1]:
        selected_stocks = f_score_df.columns[f_score_df.loc[year] >= threshold]
        selected_stocks = [stock for stock in selected_stocks if stock in returns.columns]
        if selected_stocks:
            portfolio_returns.loc[str(year+1)] = returns[selected_stocks].mean(axis=1)
    
    portfolio_cumulative_returns = (1 + portfolio_returns.dropna()).cumprod() - 1
    return portfolio_returns.dropna(), portfolio_cumulative_returns

def calculate_sharpe_ratio(returns, risk_free_rate=0.02):
    excess_returns = returns - risk_free_rate / 252  # 일일 수익률로 가정
    sharpe_ratio = np.mean(excess_returns) / np.std(excess_returns)
    return sharpe_ratio * np.sqrt(252)  # 연간 샤프비율로 변환

# 예제 사용법
data = pd.DataFrame({
    'net_income': [100, 150, 200],
    'total_assets': [1000, 1100, 1200],
    'cash_flow_operations': [120, 130, 140],
    'long_term_debt': [400, 350, 300],
    'current_assets': [500, 550, 600],
    'current_liabilities': [200, 220, 250],
    'shares_outstanding': [1000, 1000, 1000],
    'gross_margin': [400, 450, 500],
    'total_revenue': [1000, 1100, 1200]
}, index=[2018, 2019, 2020])

price_data = pd.DataFrame({
    'AAPL': [150, 155, 160, 165, 170, 175, 180],
    'MSFT': [100, 102, 104, 106, 108, 110, 112],
    'GOOGL': [1200, 1220, 1240, 1260, 1280, 1300, 1320]
}, index=pd.date_range(start='2020-01-01', periods=7, freq='Y'))

f_score_df = calculate_f_score(data)
f_score_df['AAPL'] = [6, 5, 7]
f_score_df['MSFT'] = [4, 6, 5]
f_score_df['GOOGL'] = [7, 8, 6]

portfolio_returns, portfolio_cumulative_returns = backtest_strategy(price_data, f_score_df)

# 샤프비율 계산
if not portfolio_returns.empty:
    sharpe_ratio = calculate_sharpe_ratio(portfolio_returns)
    print(f"Sharpe Ratio: {sharpe_ratio}")

print("Portfolio Cumulative Returns:")
print(portfolio_cumulative_returns)


C:\Users\JYB\AppData\Local\Temp/ipykernel_24772/3746290014.py:19: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  portfolio_returns = pd.Series(index=returns.index)


TypeError: cannot convert the series to <class 'float'>

In [5]:
import pandas as pd
import numpy as np

def calculate_f_score(data):
    f_score = pd.DataFrame(index=data.index)
    f_score['ROA'] = (data['net_income'] / data['total_assets']).apply(lambda x: 1 if x > 0 else 0)
    f_score['CFO'] = data['cash_flow_operations'].apply(lambda x: 1 if x > 0 else 0)
    f_score['change_ROA'] = (data['net_income'] / data['total_assets']).diff().apply(lambda x: 1 if x > 0 else 0)
    f_score['change_leverage'] = (data['long_term_debt'] / data['total_assets']).diff().apply(lambda x: 1 if x < 0 else 0)
    f_score['change_liquidity'] = (data['current_assets'] / data['current_liabilities']).diff().apply(lambda x: 1 if x > 0 else 0)
    f_score['equity_offering'] = data['shares_outstanding'].diff().apply(lambda x: 1 if x <= 0 else 0)
    f_score['change_margin'] = (data['gross_margin'] / data['total_revenue']).diff().apply(lambda x: 1 if x > 0 else 0)
    f_score['change_turnover'] = (data['total_revenue'] / data['total_assets']).diff().apply(lambda x: 1 if x > 0 else 0)
    f_score['F_Score'] = f_score.sum(axis=1)
    return f_score

def backtest_strategy(price_data, f_score_df, threshold=5):
    returns = price_data.pct_change().shift(-1).fillna(0)
    portfolio_returns = pd.Series(dtype=float)

    for year in f_score_df.index[:-1]:
        selected_stocks = f_score_df.columns[f_score_df.loc[year] >= threshold]
        selected_stocks = [stock for stock in selected_stocks if stock in returns.columns]
        if selected_stocks:
            # 포트폴리오 수익률을 계산하고 이를 float으로 변환
            portfolio_returns.loc[str(year+1)] = returns[selected_stocks].mean(axis=1).astype(float)
    
    portfolio_cumulative_returns = (1 + portfolio_returns.dropna()).cumprod() - 1
    return portfolio_returns.dropna(), portfolio_cumulative_returns

def calculate_sharpe_ratio(returns, risk_free_rate=0.02):
    excess_returns = returns - risk_free_rate / 252  # 일일 수익률로 가정
    sharpe_ratio = np.mean(excess_returns) / np.std(excess_returns)
    return sharpe_ratio * np.sqrt(252)  # 연간 샤프비율로 변환

# 예제 사용법
data = pd.DataFrame({
    'net_income': [100, 150, 200],
    'total_assets': [1000, 1100, 1200],
    'cash_flow_operations': [120, 130, 140],
    'long_term_debt': [400, 350, 300],
    'current_assets': [500, 550, 600],
    'current_liabilities': [200, 220, 250],
    'shares_outstanding': [1000, 1000, 1000],
    'gross_margin': [400, 450, 500],
    'total_revenue': [1000, 1100, 1200]
}, index=[2018, 2019, 2020])

price_data = pd.DataFrame({
    'AAPL': [150, 155, 160, 165, 170, 175, 180],
    'MSFT': [100, 102, 104, 106, 108, 110, 112],
    'GOOGL': [1200, 1220, 1240, 1260, 1280, 1300, 1320]
}, index=pd.date_range(start='2020-01-01', periods=7, freq='Y'))

f_score_df = calculate_f_score(data)
f_score_df['AAPL'] = [6, 5, 7]
f_score_df['MSFT'] = [4, 6, 5]
f_score_df['GOOGL'] = [7, 8, 6]

portfolio_returns, portfolio_cumulative_returns = backtest_strategy(price_data, f_score_df)

# 샤프비율 계산
if not portfolio_returns.empty:
    sharpe_ratio = calculate_sharpe_ratio(portfolio_returns)
    print(f"Sharpe Ratio: {sharpe_ratio}")

print("Portfolio Cumulative Returns:")
print(portfolio_cumulative_returns)


TypeError: cannot convert the series to <class 'float'>

In [6]:
import pandas as pd
import numpy as np

def calculate_f_score(data):
    f_score = pd.DataFrame(index=data.index)
    f_score['ROA'] = (data['net_income'] / data['total_assets']).apply(lambda x: 1 if x > 0 else 0)
    f_score['CFO'] = data['cash_flow_operations'].apply(lambda x: 1 if x > 0 else 0)
    f_score['change_ROA'] = (data['net_income'] / data['total_assets']).diff().apply(lambda x: 1 if x > 0 else 0)
    f_score['change_leverage'] = (data['long_term_debt'] / data['total_assets']).diff().apply(lambda x: 1 if x < 0 else 0)
    f_score['change_liquidity'] = (data['current_assets'] / data['current_liabilities']).diff().apply(lambda x: 1 if x > 0 else 0)
    f_score['equity_offering'] = data['shares_outstanding'].diff().apply(lambda x: 1 if x <= 0 else 0)
    f_score['change_margin'] = (data['gross_margin'] / data['total_revenue']).diff().apply(lambda x: 1 if x > 0 else 0)
    f_score['change_turnover'] = (data['total_revenue'] / data['total_assets']).diff().apply(lambda x: 1 if x > 0 else 0)
    f_score['F_Score'] = f_score.sum(axis=1)
    return f_score

def backtest_strategy(price_data, f_score_df, threshold=5):
    returns = price_data.pct_change().shift(-1).fillna(0)
    portfolio_returns = pd.Series(dtype=float)

    for year in f_score_df.index[:-1]:
        selected_stocks = f_score_df.columns[f_score_df.loc[year] >= threshold]
        selected_stocks = [stock for stock in selected_stocks if stock in returns.columns]
        if selected_stocks:
            for date in returns.index:
                portfolio_returns.at[date] = returns.loc[date, selected_stocks].mean()
    
    portfolio_cumulative_returns = (1 + portfolio_returns.dropna()).cumprod() - 1
    return portfolio_returns.dropna(), portfolio_cumulative_returns

def calculate_sharpe_ratio(returns, risk_free_rate=0.02):
    excess_returns = returns - risk_free_rate / 252  # 일일 수익률로 가정
    sharpe_ratio = np.mean(excess_returns) / np.std(excess_returns)
    return sharpe_ratio * np.sqrt(252)  # 연간 샤프비율로 변환

# 예제 사용법
data = pd.DataFrame({
    'net_income': [100, 150, 200],
    'total_assets': [1000, 1100, 1200],
    'cash_flow_operations': [120, 130, 140],
    'long_term_debt': [400, 350, 300],
    'current_assets': [500, 550, 600],
    'current_liabilities': [200, 220, 250],
    'shares_outstanding': [1000, 1000, 1000],
    'gross_margin': [400, 450, 500],
    'total_revenue': [1000, 1100, 1200]
}, index=[2018, 2019, 2020])

price_data = pd.DataFrame({
    'AAPL': [150, 155, 160, 165, 170, 175, 180],
    'MSFT': [100, 102, 104, 106, 108, 110, 112],
    'GOOGL': [1200, 1220, 1240, 1260, 1280, 1300, 1320]
}, index=pd.date_range(start='2020-01-01', periods=7, freq='Y'))

f_score_df = calculate_f_score(data)
f_score_df['AAPL'] = [6, 5, 7]
f_score_df['MSFT'] = [4, 6, 5]
f_score_df['GOOGL'] = [7, 8, 6]

portfolio_returns, portfolio_cumulative_returns = backtest_strategy(price_data, f_score_df)

# 샤프비율 계산
if not portfolio_returns.empty:
    sharpe_ratio = calculate_sharpe_ratio(portfolio_returns)
    print(f"Sharpe Ratio: {sharpe_ratio}")

print("Portfolio Cumulative Returns:")
print(portfolio_cumulative_returns)


Sharpe Ratio: 38.49781619129201
Portfolio Cumulative Returns:
2020-12-31    0.023333
2021-12-31    0.046617
2022-12-31    0.069856
2023-12-31    0.093052
2024-12-31    0.116208
2025-12-31    0.139328
2026-12-31    0.139328
dtype: float64
